In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from functions_julieta import *
import nltk
from nltk.tokenize import word_tokenize
import os

# STOPWORDS
nltk.download('punkt')

with open('spanish.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

spanish_stopwords = [line.strip() for line in lines]

# Parámetros
SAVE_CSV = True
SAVE_TXT = True
caso = 'julieta'

# Dataframes
df = pd.read_csv(f'{caso}.csv')
df_chat = pd.read_csv(f'{caso}_chat.csv')

[nltk_data] Downloading package punkt to C:\Users\Daniel Minaya
[nltk_data]     Vargas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Limpieza de datos 

In [2]:
# Secciones
df.seccion.unique()

array([1, 4, 3, 7, 8, 9, 6, 2], dtype=int64)

In [3]:
# Eliminamos fases extras
clean_df = delete_extra_phases(df)

# Estandarizamos titulos y arreglamos etapa
clean_df = std_titles(clean_df)

# Pasamos tiempo de str a datetime
clean_df['time'] = clean_df['time'].apply(to_datetime)

clean_df.sample(5)

,id,user_id,team_id,gender,df,title,opt_left,opt_right,max_num,sel,comment,phase,time,curso,seccion,etapa
1714,12732,2857,1488.0,M,1,[Grup] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,3,debido a la ambiguedad del problema decidimos ...,5,2021-05-24 18:45:23.147,2021.CD1100,9,Grup
1089,13674,3019,1502.0,F,1,[Grup] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,4,"Al final, Julieta tiene las herramientas para ...",5,2021-05-25 23:52:18.473,2021.CD1100,8,Grup
1991,15364,3145,1586.0,F,1,[Grup] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,3,No debería usar la información porque no es c...,4,2021-05-26 17:44:03.693,2021.CD1100,2,Grup
1538,14500,3314,NaN,M,1,"[Ind2] Después de la discusión en grupo, ¿cómo...",Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,1,"copiar esta sobre valorado, se ve como un peca...",5,2021-05-26 01:37:40.681,2021.CD1100,1,Ind2
1795,12810,2540,NaN,M,1,"[Ind2] Después de la discusión en grupo, ¿cómo...",Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,2,"mi posición si cambió, ya que no había tenido ...",6,2021-05-24 18:51:59.087,2021.CD1100,9,Ind2


In [4]:
# Eliminamos repeticiones (al estar ordenadas nos quedamos con la primera realización)
no_dup = clean_df.sort_values('time').drop_duplicates(subset=['user_id', 'title', 'seccion'])
no_dup.sample(5)

,id,user_id,team_id,gender,df,title,opt_left,opt_right,max_num,sel,comment,phase,time,curso,seccion,etapa
2056,15498,3110,NaN,F,1,"[Ind2] Después de la discusión en grupo, ¿cómo...",Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,1,"mi posición sigue siendo la misma, ya que tien...",5,2021-05-26 17:49:41.275,2021.CD1100,2,Ind2
415,15769,2641,1651.0,M,1,[Grup] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,4,Pongo 4 puesto que considero que uno estudia p...,4,2021-05-26 18:17:01.265,2021.CD1100,3,Grup
605,13655,3231,NaN,M,1,[Ind1] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,6,Personalmente creo que no debería utilizar la ...,3,2021-05-25 23:45:15.054,2021.CD1100,7,Ind1
438,15794,3035,1650.0,M,1,[Grup] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,3,"No creo que la copia esté justificada, sin emb...",4,2021-05-26 18:23:01.877,2021.CD1100,3,Grup
2087,15567,3386,NaN,M,1,"[Ind2] Después de la discusión en grupo, ¿cómo...",Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,3,Pues sigo considerando que ayudarse de esa man...,5,2021-05-26 17:51:32.065,2021.CD1100,2,Ind2


In [5]:
# Separamos por secciones
group = no_dup.groupby('seccion')

dict_df = {section: dataframe for section, dataframe in group}

In [6]:
# Número de fases completas por alumnos (Antes de eliminar)
for section in dict_df.keys():
    print(f"Sección {section}:", np.unique(dict_df[section].groupby('user_id').count().id, return_counts=True))

Sección 1: (array([2, 3], dtype=int64), array([ 1, 99], dtype=int64))
Sección 2: (array([1, 2, 3], dtype=int64), array([ 3,  6, 80], dtype=int64))
Sección 3: (array([2, 3], dtype=int64), array([ 3, 92], dtype=int64))
Sección 4: (array([1, 2, 3], dtype=int64), array([ 5,  5, 90], dtype=int64))
Sección 6: (array([1, 2, 3], dtype=int64), array([ 3,  2, 78], dtype=int64))
Sección 7: (array([1, 3], dtype=int64), array([ 1, 91], dtype=int64))
Sección 8: (array([1, 2, 3], dtype=int64), array([ 1,  7, 81], dtype=int64))
Sección 9: (array([2, 3], dtype=int64), array([  3, 126], dtype=int64))


In [7]:
# Eliminamos los alumnos con alguna fase incompleta por seccion
for section in dict_df.keys():
    mask = dict_df[section].groupby('user_id').id.transform('count') == 3
    dict_df[section] = dict_df[section][mask]

In [8]:
# Número de fases completas por alumnos (Despues de eliminar)
for section in dict_df.keys():
    print(f"Sección {section}:", np.unique(dict_df[section].groupby('user_id').count().id, return_counts=True))

Sección 1: (array([3], dtype=int64), array([99], dtype=int64))
Sección 2: (array([3], dtype=int64), array([80], dtype=int64))
Sección 3: (array([3], dtype=int64), array([92], dtype=int64))
Sección 4: (array([3], dtype=int64), array([90], dtype=int64))
Sección 6: (array([3], dtype=int64), array([78], dtype=int64))
Sección 7: (array([3], dtype=int64), array([91], dtype=int64))
Sección 8: (array([3], dtype=int64), array([81], dtype=int64))
Sección 9: (array([3], dtype=int64), array([126], dtype=int64))


In [9]:
# Juntamos todo el dataframe
clean_df = pd.concat(dict_df.values(), ignore_index=True)
clean_df.sample(5)

,id,user_id,team_id,gender,df,title,opt_left,opt_right,max_num,sel,comment,phase,time,curso,seccion,etapa
49,14280,2881,NaN,F,1,[Ind1] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,4,Confiar en sus conocimientos para hacer el con...,3,2021-05-26 01:02:40.848,2021.CD1100,1,Ind1
31,14256,2701,NaN,F,1,[Ind1] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,4,lo que Julieta debería hacer seria primero res...,3,2021-05-26 01:01:31.787,2021.CD1100,1,Ind1
1822,13835,3195,NaN,M,1,"[Ind2] Después de la discusión en grupo, ¿cómo...",Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,4,después de la discusión cambie un poco de opin...,6,2021-05-26 00:00:45.919,2021.CD1100,8,Ind2
2051,12795,2628,NaN,M,1,"[Ind2] Después de la discusión en grupo, ¿cómo...",Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,5,Es verdad que existen muchas variables para co...,6,2021-05-24 18:51:12.417,2021.CD1100,9,Ind2
599,15621,2771,NaN,F,1,[Ind1] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,3,"Debería utilizar la información, más aun si la...",3,2021-05-26 17:54:06.372,2021.CD1100,3,Ind1


## Chat

In [10]:
# Vemos entradas con team_id nulo
df_chat[df_chat.team_id.isna()]

,id,user_id,team_id,gender,df,title,opt_left,opt_right,message,phase,time,reply_to,curso,seccion
1844,19707,2868,NaN,M,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,hola,5,2021-05-25T23:35:55.345Z,NaN,2021.CD1100,8
1931,19794,2868,NaN,M,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,hola,5,2021-05-25T23:36:28.792Z,NaN,2021.CD1100,8
2044,19907,2868,NaN,M,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,klhjh,5,2021-05-25T23:37:16.374Z,NaN,2021.CD1100,8
2053,19916,2868,NaN,M,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,hola,5,2021-05-25T23:37:21.667Z,NaN,2021.CD1100,8
3414,30277,2974,NaN,F,1,"[Acordar entre todas/os la respuesta, discutie...",Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,holaaa,4,2021-05-28T01:29:09.381Z,NaN,2021.CD1100,6
3458,30321,2974,NaN,F,1,"[Acordar entre todas/os la respuesta, discutie...",Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,pongamos 6,4,2021-05-28T01:29:35.471Z,NaN,2021.CD1100,6
3473,30336,3447,NaN,M,1,"[Acordar entre todas/os la respuesta, discutie...",Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,hola?,4,2021-05-28T01:29:46.677Z,NaN,2021.CD1100,6
12549,26386,3396,NaN,M,1,[Proporcione argumentos para justificar su res...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,Hay alguien?,4,2021-05-26T17:31:59.995Z,NaN,2021.CD1100,2
12853,27274,3396,NaN,M,1,[Proporcione argumentos para justificar su res...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,V,4,2021-05-26T17:36:23.791Z,NaN,2021.CD1100,2


In [11]:
# No aportan información, así que las eliminamos
df_chat = df_chat[~df_chat.team_id.isna()]
df_chat.head()

,id,user_id,team_id,gender,df,title,opt_left,opt_right,message,phase,time,reply_to,curso,seccion
0,25846,2953,1624.0,M,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,hola,4,2021-05-26T17:29:51.667Z,NaN,2021.CD1100,4
1,25849,2926,1612.0,F,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,hola,4,2021-05-26T17:29:53.374Z,NaN,2021.CD1100,4
2,25850,2787,1642.0,F,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,holaa equipoo,4,2021-05-26T17:29:53.453Z,NaN,2021.CD1100,4
3,25853,3171,1640.0,M,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,Holaaaaaaa,4,2021-05-26T17:29:55.042Z,NaN,2021.CD1100,4
4,25858,2790,1642.0,F,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,holi?,4,2021-05-26T17:29:56.538Z,NaN,2021.CD1100,4


# Limpieza de texto

## Individual

In [12]:
# Pasamos a minúsculas las justificaciones
clean_df['comment'] = clean_df['comment'].str.lower()

# Quitamos stopwords de las justificaciones
def remove_stopwords(text):
    words = word_tokenize(text, language='spanish')
    filtered_words = [word for word in words if word.lower() not in spanish_stopwords]
    cleaned_text = ' '.join(filtered_words)
    return cleaned_text

clean_df['comment'] = clean_df['comment'].apply(remove_stopwords)

clean_df.head()

,id,user_id,team_id,gender,df,title,opt_left,opt_right,max_num,sel,comment,phase,time,curso,seccion,etapa
0,14218,2806,NaN,F,1,[Ind1] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,6,", sumario reputación .",3,2021-05-26 00:58:42.474,2021.CD1100,1,Ind1
1,14219,2889,NaN,F,1,[Ind1] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,6,quizá podría ayudar instante futuro afectará,3,2021-05-26 00:58:48.887,2021.CD1100,1,Ind1
2,14220,2897,NaN,F,1,[Ind1] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,3,utilizar información whatsapp sólo si ve apuros .,3,2021-05-26 00:58:52.908,2021.CD1100,1,Ind1
3,14221,2814,NaN,M,1,[Ind1] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,3,podria hacerlo sola comparar respuestas asegur...,3,2021-05-26 00:59:02.265,2021.CD1100,1,Ind1
4,14222,3334,NaN,M,1,[Ind1] Julieta en esta situación a la que se v...,Usar información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,6,1,"insisto , recursos disponibles",3,2021-05-26 00:59:05.371,2021.CD1100,1,Ind1


## Chat

In [13]:
# Pasamos los mensajes a minúsculas
df_chat['message'] = df_chat['message'].str.lower()
df_chat.head()

,id,user_id,team_id,gender,df,title,opt_left,opt_right,message,phase,time,reply_to,curso,seccion
0,25846,2953,1624.0,M,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,hola,4,2021-05-26T17:29:51.667Z,NaN,2021.CD1100,4
1,25849,2926,1612.0,F,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,hola,4,2021-05-26T17:29:53.374Z,NaN,2021.CD1100,4
2,25850,2787,1642.0,F,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,holaa equipoo,4,2021-05-26T17:29:53.453Z,NaN,2021.CD1100,4
3,25853,3171,1640.0,M,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,holaaaaaaa,4,2021-05-26T17:29:55.042Z,NaN,2021.CD1100,4
4,25858,2790,1642.0,F,1,"[Acordar entre todas/os la respuesta, discutie...",Usar la información del grupo de WhatsApp,No usar la información del grupo de WhatsApp,holi?,4,2021-05-26T17:29:56.538Z,NaN,2021.CD1100,4


# Guardado

In [14]:
if SAVE_CSV:
    clean_df.to_csv('clean_julieta.csv', index=False)

# Agrupamos por grupo
group = df_chat.groupby('team_id')

# Exportamos cada conversación grupal en txt
if SAVE_TXT:
    for team_id, dataframe in group:
        chat_messages = [f"{row['user_id']}: {row['message']}" for _, row in dataframe.iterrows()]

        filename = os.path.join(f'chat_{caso}', f"team_{team_id}_chat.txt")

        with open(filename, 'w', encoding='utf-8') as file:
            file.write("\n".join(chat_messages))